In [ ]:
import openai
import os
from datetime import datetime
import json
from pathlib import Path

# PDF generation
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, PageBreak
from reportlab.lib.units import inch
from reportlab.lib.enums import TA_JUSTIFY, TA_CENTER

class BibleLanguageLearningSystem:
    """
    Agentic AI system for language learning through Bible study.
    Uses multiple specialized agents to create comprehensive lessons.
    Uses OpenAI's GPT models.
    """
    
    def __init__(self, api_key: str, target_language: str = "Spanish", model: str = "gpt-4o"):
        self.client = openai.OpenAI(api_key=api_key)
        self.model = model  # Options: gpt-4o, gpt-4o-mini, gpt-4-turbo, gpt-3.5-turbo
        self.target_language = target_language
        self.conversation_history = []
        
    def _call_gpt(self, system_prompt: str, user_message: str, temperature: float = 1.0) -> str:
        """Helper method to call OpenAI API"""
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_message}
            ],
            temperature=temperature,
            max_tokens=4000
        )
        return response.choices[0].message.content
    
    def agent_verse_retriever(self, language_level: str) -> dict:
        """
        Agent 1: Retrieves verse of the day and meditation paragraph
        """
        system_prompt = f"""You are a Bible study coordinator agent. Your role is to:
1. Select an appropriate verse of the day (Verse of the Day)
2. Provide a short meditation paragraph (2-3 sentences) about that verse in both English and {self.target_language}
3. Ensure the content is appropriate for {language_level} language learners

Return your response in valid JSON format with these exact keys:
- "verse_reference": The Bible verse reference (e.g., "John 3:16")
- "verse_text_english": The verse in English
- "verse_text_{self.target_language.lower()}": The verse in {self.target_language}
- "meditation_english": Meditation paragraph in English
- "meditation_{self.target_language.lower()}": Meditation paragraph in {self.target_language}

Return ONLY the JSON object, no additional text."""

        user_message = f"Please provide a verse of the day with meditation for {self.target_language} language learning at {language_level} level."
        
        response = self._call_gpt(system_prompt, user_message, temperature=0.7)
        
        # Parse JSON response
        try:
            # Extract JSON from response if it contains markdown code blocks
            if "```json" in response:
                json_str = response.split("```json")[1].split("```")[0].strip()
            elif "```" in response:
                json_str = response.split("```")[1].split("```")[0].strip()
            else:
                json_str = response.strip()
            
            return json.loads(json_str)
        except Exception as e:
            print(f"Warning: JSON parsing failed: {e}")
            # Fallback if JSON parsing fails
            return {
                "verse_reference": "John 3:16",
                "verse_text_english": "For God so loved the world that he gave his one and only Son...",
                "verse_text_spanish": "Porque tanto amó Dios al mundo que dio a su Hijo unigénito...",
                "meditation_english": response[:200] if len(response) > 200 else "God's love for humanity.",
                "meditation_spanish": "El amor de Dios por la humanidad."
            }
    
    def agent_content_creator(self, verse_data: dict, language_level: str) -> dict:
        """
        Agent 2: Creates reading comprehension paragraph based on the verse
        """
        system_prompt = f"""You are a language learning content creator specialized in Bible-based materials.
Create a reading comprehension paragraph (150-200 words) in {self.target_language} based on the provided verse.

Requirements:
- Appropriate for {language_level} level ({self._get_level_description(language_level)})
- Include theological insights and practical applications
- Use clear, educational language
- Also provide English translation

Return valid JSON with these exact keys:
- "reading_text_{self.target_language.lower()}": The reading text in {self.target_language}
- "reading_text_english": The reading text in English
- "key_vocabulary": Array of important vocabulary words from the text

Return ONLY the JSON object, no additional text."""

        user_message = f"""Create reading comprehension content based on:
Verse: {verse_data.get('verse_reference', 'N/A')}
Text: {verse_data.get(f'verse_text_{self.target_language.lower()}', verse_data.get('verse_text_english', ''))}
Meditation: {verse_data.get(f'meditation_{self.target_language.lower()}', verse_data.get('meditation_english', ''))}"""

        response = self._call_gpt(system_prompt, user_message, temperature=0.8)
        
        try:
            if "```json" in response:
                json_str = response.split("```json")[1].split("```")[0].strip()
            elif "```" in response:
                json_str = response.split("```")[1].split("```")[0].strip()
            else:
                json_str = response.strip()
            
            return json.loads(json_str)
        except Exception as e:
            print(f"Warning: JSON parsing failed: {e}")
            return {
                "reading_text": response[:500],
                "key_vocabulary": ["amor", "Dios", "salvación"]
            }
    
    def agent_lesson_designer(self, verse_data: dict, reading_data: dict, language_level: str) -> dict:
        """
        Agent 3: Designs comprehensive lesson with all skill areas
        """
        system_prompt = f"""You are an expert language lesson designer for {self.target_language}.
Create a comprehensive lesson for {language_level} level including:

1. READING SECTION: 4-5 comprehension questions about the reading text (in {self.target_language})
2. WRITING SECTION: 3 writing prompts/exercises (in {self.target_language})
3. LISTENING SECTION: 4 questions simulating audio comprehension (in {self.target_language})
4. SPEAKING SECTION: 3 speaking prompts/discussion questions (in {self.target_language})
5. FILLING SECTION: 10 filling exercises about the reading using key words for {language_level} (in {self.target_language})

All questions should relate to the Bible content and test understanding of both language AND theology.

Return valid JSON with these exact keys:
- "reading_exercises": Array of objects with "question" field
- "writing_exercises": Array of objects with "question" field
- "listening_exercises": Array of objects with "question" field
- "speaking_exercises": Array of objects with "question" field
- "filling_exercises": Array of objects with "question" field

Return ONLY the JSON object, no additional text."""

        user_message = f"""Design lesson based on:
Verse: {verse_data.get('verse_reference')}
Reading Text: {reading_data.get(f'reading_text_{self.target_language.lower()}', reading_data.get('reading_text', ''))}
Key Vocabulary: {reading_data.get('key_vocabulary', [])}"""

        response = self._call_gpt(system_prompt, user_message, temperature=0.7)
        
        try:
            if "```json" in response:
                json_str = response.split("```json")[1].split("```")[0].strip()
            elif "```" in response:
                json_str = response.split("```")[1].split("```")[0].strip()
            else:
                json_str = response.strip()
            
            return json.loads(json_str)
        except Exception as e:
            print(f"Warning: JSON parsing failed: {e}")
            return {
                "reading_exercises": [{"question": "¿Cuál es el tema principal del texto?"}],
                "writing_exercises": [{"question": "Escribe un párrafo sobre el amor de Dios."}],
                "listening_exercises": [{"question": "¿Qué palabra clave escuchaste?"}],
                "speaking_exercises": [{"question": "Describe tu relación con Dios."}],
                "filling_exercises": [{"question":"tema del verso"}]
            }
    
    def agent_answer_key_generator(self, lesson_data: dict, verse_data: dict, reading_data: dict) -> dict:
        """
        Agent 4: Generates comprehensive answer key
        """
        system_prompt = f"""You are an answer key generator for {self.target_language} language learning materials.
Provide detailed answers, explanations, and sample responses for all exercises in {self.target_language}.

For each exercise type:
- Reading: Provide correct answers with explanations
- Writing: Provide model responses (3-4 sentences each)
- Listening: Provide correct answers with key points
- Speaking: Provide model responses and discussion points
- Filling: Provide correct answers with the correct word.

Return valid JSON with these exact keys matching the exercise structure:
- "reading_exercises": Array of objects with "answer" and "explanation" fields
- "writing_exercises": Array of objects with "answer" and "explanation" fields
- "listening_exercises": Array of objects with "answer" and "explanation" fields
- "speaking_exercises": Array of objects with "answer" and "explanation" fields
- "filling_exercises" : Array of objects with "answer" and "explanation" fields

Return ONLY the JSON object, no additional text."""

        user_message = f"""Generate answer key for:
Lesson Exercises: {json.dumps(lesson_data, ensure_ascii=False)}
Verse Context: {verse_data.get('verse_reference')}
Reading Text: {reading_data.get(f'reading_text_{self.target_language.lower()}', '')}"""

        response = self._call_gpt(system_prompt, user_message, temperature=0.5)
        
        try:
            if "```json" in response:
                json_str = response.split("```json")[1].split("```")[0].strip()
            elif "```" in response:
                json_str = response.split("```")[1].split("```")[0].strip()
            else:
                json_str = response.strip()
            
            return json.loads(json_str)
        except Exception as e:
            print(f"Warning: JSON parsing failed: {e}")
            return {
                "reading_exercises": [{"answer": "El amor de Dios", "explanation": "Basado en el versículo."}],
                "writing_exercises": [{"answer": "Dios nos ama incondicionalmente...", "explanation": "Respuesta modelo."}],
                "listening_exercises": [{"answer": "amor, salvación", "explanation": "Palabras clave."}],
                "speaking_exercises": [{"answer": "Mi relación con Dios es...", "explanation": "Respuesta personal."}],
                "filling_exercises": [{"answer": "Mi relación con Dios es...", "explanation": "Respuesta sugerida."}]
            }
    
    def _get_level_description(self, level: str) -> str:
        """Get CEFR level description"""
        descriptions = {
            "A1": "Beginner - basic phrases and simple sentences",
            "A2": "Elementary - simple everyday expressions",
            "B1": "Intermediate - clear standard language on familiar matters",
            "B2": "Upper Intermediate - complex text and abstract topics",
            "C1": "Advanced - implicit meaning and fluent expression",
            "C2": "Proficient - precise, nuanced communication"
        }
        return descriptions.get(level, "Intermediate level")
    
    def generate_pdf(self, lesson_content: dict, filename: str = None):
        """
        Generate PDF with all lesson content using reportlab (free library)
        """
        if filename is None:
            filename = f"bible_lesson_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pdf"
        
        # Ensure output directory exists
        Path("lessons").mkdir(exist_ok=True)
        filepath = f"lessons/{filename}"
        
        doc = SimpleDocTemplate(filepath, pagesize=letter,
                                rightMargin=72, leftMargin=72,
                                topMargin=72, bottomMargin=18)
        
        # Container for the 'Flowable' objects
        elements = []
        
        # Define styles
        styles = getSampleStyleSheet()
        title_style = ParagraphStyle(
            'CustomTitle',
            parent=styles['Heading1'],
            fontSize=24,
            textColor='darkblue',
            spaceAfter=30,
            alignment=TA_CENTER
        )
        heading_style = styles['Heading2']
        normal_style = styles['BodyText']
        normal_style.alignment = TA_JUSTIFY
        
        # Title
        title = Paragraph(f"Bible Language Learning Lesson<br/>{self.target_language}", title_style)
        elements.append(title)
        elements.append(Spacer(1, 0.2*inch))
        
        # Date and Level
        date_text = f"Date: {datetime.now().strftime('%B %d, %Y')}<br/>Level: {lesson_content.get('level', 'B1')}<br/>Powered by: {self.model}"
        elements.append(Paragraph(date_text, normal_style))
        elements.append(Spacer(1, 0.3*inch))
        
        # Verse of the Day
        elements.append(Paragraph("📖 Verse of the Day", heading_style))
        elements.append(Spacer(1, 0.1*inch))
        
        verse_data = lesson_content.get('verse_data', {})
        verse_ref = verse_data.get('verse_reference', 'N/A')
        elements.append(Paragraph(f"<b>{verse_ref}</b>", normal_style))
        
        verse_text_key = f'verse_text_{self.target_language.lower()}'
        verse_text = verse_data.get(verse_text_key, verse_data.get('verse_text_english', 'N/A'))
        elements.append(Paragraph(f"<i>{verse_text}</i>", normal_style))
        elements.append(Spacer(1, 0.2*inch))
        
        # Meditation
        meditation_key = f'meditation_{self.target_language.lower()}'
        meditation = verse_data.get(meditation_key, verse_data.get('meditation_english', ''))
        if meditation:
            elements.append(Paragraph(f"<b>Meditación / Meditation:</b> {meditation}", normal_style))
            elements.append(Spacer(1, 0.3*inch))
        
        # Reading Text
        elements.append(Paragraph("📚 Reading Comprehension / Comprensión de Lectura", heading_style))
        elements.append(Spacer(1, 0.1*inch))
        
        reading_data = lesson_content.get('reading_data', {})
        reading_text_key = f'reading_text_{self.target_language.lower()}'
        reading_text = reading_data.get(reading_text_key, reading_data.get('reading_text', 'N/A'))
        elements.append(Paragraph(reading_text, normal_style))
        elements.append(Spacer(1, 0.3*inch))
        
        # Key Vocabulary
        vocab = reading_data.get('key_vocabulary', [])
        if vocab and isinstance(vocab, list) and len(vocab) > 0:
            elements.append(Paragraph("<b>Key Vocabulary / Vocabulario Clave:</b>", normal_style))
            vocab_text = ", ".join(vocab) if isinstance(vocab[0], str) else str(vocab)
            elements.append(Paragraph(vocab_text, normal_style))
            elements.append(Spacer(1, 0.3*inch))
        
        elements.append(PageBreak())
        
        # Exercises
        lesson_data = lesson_content.get('lesson_data', {})
        
        # Reading Exercises
        self._add_exercise_section(elements, "📖 Reading Exercises / Ejercicios de Lectura", 
                                   lesson_data.get('reading_exercises', []),
                                   styles)
        
        # Writing Exercises
        self._add_exercise_section(elements, "✍️ Writing Exercises / Ejercicios de Escritura", 
                                   lesson_data.get('writing_exercises', []),
                                   styles)
        
        # Listening Exercises
        self._add_exercise_section(elements, "👂 Listening Exercises / Ejercicios de Escucha", 
                                   lesson_data.get('listening_exercises', []),
                                   styles)
        
        # Speaking Exercises
        self._add_exercise_section(elements, "🗣️ Speaking Exercises / Ejercicios de Conversación", 
                                   lesson_data.get('speaking_exercises', []),
                                   styles)
        
        # Filling Exercises
        self._add_exercise_section(elements, "🗣️ Filling Exercises / Ejercicios de llenar", 
                                   lesson_data.get('filling_exercises', []),
                                   styles)
        
        elements.append(PageBreak())
        
        # Answer Key
        elements.append(Paragraph("✅ Answer Key / Clave de Respuestas", heading_style))
        elements.append(Spacer(1, 0.2*inch))
        
        answers = lesson_content.get('answers', {})
        self._add_answers_section(elements, "Reading Answers", 
                                 answers.get('reading_exercises', []),
                                 styles)
        self._add_answers_section(elements, "Writing Answers", 
                                 answers.get('writing_exercises', []),
                                 styles)
        self._add_answers_section(elements, "Listening Answers", 
                                 answers.get('listening_exercises', []),
                                 styles)
        self._add_answers_section(elements, "Speaking Answers", 
                                 answers.get('speaking_exercises', []),
                                 styles)
        self._add_answers_section(elements, "Filling Answers", 
                                 answers.get('filling_exercises', []),
                                 styles)
        
        # Build PDF
        doc.build(elements)
        print(f"\n✅ PDF generated successfully: {filepath}")
        return filepath
    
    def _add_exercise_section(self, elements, title, exercises, styles):
        """Helper to add exercise section to PDF"""
        elements.append(Paragraph(title, styles['Heading2']))
        elements.append(Spacer(1, 0.1*inch))
        
        if isinstance(exercises, list):
            for i, ex in enumerate(exercises, 1):
                if isinstance(ex, dict):
                    question = ex.get('question', str(ex))
                else:
                    question = str(ex)
                elements.append(Paragraph(f"{i}. {question}", styles['BodyText']))
                elements.append(Spacer(1, 0.15*inch))
        else:
            elements.append(Paragraph(str(exercises), styles['BodyText']))
        
        elements.append(Spacer(1, 0.3*inch))
    
    def _add_answers_section(self, elements, title, answers, styles):
        """Helper to add answers section to PDF"""
        elements.append(Paragraph(f"<b>{title}</b>", styles['Heading3']))
        elements.append(Spacer(1, 0.1*inch))
        
        if isinstance(answers, list):
            for i, ans in enumerate(answers, 1):
                if isinstance(ans, dict):
                    answer_text = ans.get('answer', str(ans))
                    explanation = ans.get('explanation', '')
                    text = f"{i}. {answer_text}"
                    if explanation:
                        text += f" <i>({explanation})</i>"
                else:
                    text = f"{i}. {str(ans)}"
                elements.append(Paragraph(text, styles['BodyText']))
                elements.append(Spacer(1, 0.1*inch))
        else:
            elements.append(Paragraph(str(answers), styles['BodyText']))
        
        elements.append(Spacer(1, 0.2*inch))
    
    def run_full_lesson_generation(self, language_level: str = "B1"):
        """
        Orchestrates all agents to generate a complete lesson
        """
        print(f"\n🚀 Starting Bible Language Learning System")
        print(f"AI Model: {self.model}")
        print(f"Target Language: {self.target_language}")
        print(f"Level: {language_level}\n")
        
        # Step 1: Get verse and meditation
        print("📖 Agent 1: Retrieving verse of the day...")
        verse_data = self.agent_verse_retriever(language_level)
        print(f"   ✓ Retrieved: {verse_data.get('verse_reference', 'N/A')}")
        
        # Step 2: Create reading comprehension
        print("\n📚 Agent 2: Creating reading comprehension content...")
        reading_data = self.agent_content_creator(verse_data, language_level)
        print(f"   ✓ Created reading text")
        
        # Step 3: Design lesson
        print("\n🎓 Agent 3: Designing comprehensive lesson...")
        lesson_data = self.agent_lesson_designer(verse_data, reading_data, language_level)
        print(f"   ✓ Designed lesson with all skill areas")
        
        # Step 4: Generate answers
        print("\n✅ Agent 4: Generating answer key...")
        answers = self.agent_answer_key_generator(lesson_data, verse_data, reading_data)
        print(f"   ✓ Generated comprehensive answers")
        
        # Step 5: Generate PDF
        print("\n📄 Generating PDF...")
        lesson_content = {
            'level': language_level,
            'verse_data': verse_data,
            'reading_data': reading_data,
            'lesson_data': lesson_data,
            'answers': answers
        }
        
        pdf_path = self.generate_pdf(lesson_content)
        
        print("\n✨ Lesson generation complete!")
        return lesson_content, pdf_path




In [9]:
def main():
    """
    Main function to run the Bible Language Learning System
    """
    print("=" * 60)
    print("Bible Language Learning System (OpenAI/ChatGPT Version)")
    print("An Agentic AI Approach to Language Education")
    print("=" * 60)
    
    from dotenv import load_dotenv

    # Load environment variables from .env file
    load_dotenv()
    api_key = os.getenv("OPENAI_API_KEY")
    
    # Ask for language level
    print("\n📊 Language Levels Available:")
    print("  A1 - Beginner")
    print("  A2 - Elementary")
    print("  B1 - Intermediate (default)")
    print("  B2 - Upper Intermediate")
    print("  C1 - Advanced")
    print("  C2 - Proficient")
    
    level = input("\nEnter your language level (or press Enter for B1): ").strip().upper()
    if not level or level not in ['A1', 'A2', 'B1', 'B2', 'C1', 'C2']:
        level = 'B1'
        print(f"Using default level: {level}")
    
    # Ask for target language
    target_lang = input("\nEnter target language (default: Spanish): ").strip()
    if not target_lang:
        target_lang = "Spanish"
    
    # Ask for model
    print("\n🤖 Available Models:")
    print("  1. gpt-4o (most capable, recommended)")
    print("  2. gpt-4o-mini (faster, more affordable)")
    print("  3. gpt-4-turbo")
    print("  4. gpt-3.5-turbo (cheapest)")
    
    model_choice = input("\nSelect model (1-4, default: 1): ").strip()
    model_map = {
        "1": "gpt-4o",
        "2": "gpt-4o-mini",
        "3": "gpt-4-turbo",
        "4": "gpt-3.5-turbo",
        "": "gpt-4o"
    }
    model = model_map.get(model_choice, "gpt-4o")
    print(f"Using model: {model}")
    
    # Initialize system
    system = BibleLanguageLearningSystem(
        api_key=api_key, 
        target_language=target_lang,
        model=model
    )
    
    # Run lesson generation
    lesson_content, pdf_path = system.run_full_lesson_generation(language_level=level)
    
    # Display summary
    print("\n" + "=" * 60)
    print("📋 LESSON SUMMARY")
    print("=" * 60)
    print(f"Verse: {lesson_content['verse_data'].get('verse_reference', 'N/A')}")
    print(f"Language: {target_lang}")
    print(f"Level: {level}")
    print(f"Model: {model}")
    print(f"PDF Location: {pdf_path}")
    print("\n✅ Your personalized Bible language lesson is ready!")
    print("=" * 60)



In [10]:

if __name__ == "__main__":
    main()

Bible Language Learning System (OpenAI/ChatGPT Version)
An Agentic AI Approach to Language Education

📊 Language Levels Available:
  A1 - Beginner
  A2 - Elementary
  B1 - Intermediate (default)
  B2 - Upper Intermediate
  C1 - Advanced
  C2 - Proficient

🤖 Available Models:
  1. gpt-4o (most capable, recommended)
  2. gpt-4o-mini (faster, more affordable)
  3. gpt-4-turbo
  4. gpt-3.5-turbo (cheapest)
Using model: gpt-4o-mini

🚀 Starting Bible Language Learning System
AI Model: gpt-4o-mini
Target Language: English
Level: A1

📖 Agent 1: Retrieving verse of the day...
   ✓ Retrieved: Philippians 4:13

📚 Agent 2: Creating reading comprehension content...
   ✓ Created reading text

🎓 Agent 3: Designing comprehensive lesson...
   ✓ Designed lesson with all skill areas

✅ Agent 4: Generating answer key...
   ✓ Generated comprehensive answers

📄 Generating PDF...

✅ PDF generated successfully: lessons/bible_lesson_20260127_185456.pdf

✨ Lesson generation complete!

📋 LESSON SUMMARY
Verse: Ph